# Merge thermo libraries

### Necessary packages

In [ ]:
import logging
import os

from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase

from toolbox.base import read_species_from_yml, write_species_to_yml
from toolbox.thermolib import find_thermo_libs, read_thermo_lib_by_path, \
                              merge_thermo_lib, draw_free_energies

%matplotlib inline
%load_ext autoreload
%autoreload 2

logger = logging.getLogger()
logger.setLevel(logging.INFO)

### [OPTIONAL] Assign a log file to record all the changes 

In [ ]:
fh = logging.FileHandler('thermo_lib_tools.log', mode="a+")
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
logger.addHandler(fh)

### Load a RMG database instance
You may need to add some RMG built-in libraries for your purpose. The generated `thermo_database` will be used in the later section.

In [ ]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'],
    thermo_libraries = [],  # Can add other library if necessary
    kinetics_families = "default",
    reaction_libraries = [],
    kinetics_depositories = ['training'],
)

thermo_database = database.thermo

### Input
Merge the libraries from working directory into the base library
- `base_thermo_lib` (str): the full path to an existing RMG thermolibrary.
- `work_dir` (str): indicate where your thermo libraries (generated by ARC). Do not need to provide the full path to the file because the script will search for the thermolibrary.

In [ ]:
base_thermo_lib = ''
work_dir = ''

In [ ]:
# Get the base thermo library
read_thermo_lib_by_path(base_thermo_lib, thermo_database)
base_lib = thermo_database.libraries[base_thermo_lib]

# Get the thermo libraries under work_dir
thermo_lib_list = find_thermo_libs(work_dir)
for thermo_lib in thermo_lib_list:
    read_thermo_lib_by_path(thermo_lib, thermo_database)

# Combine the thermo libraries 
for thermo_lib in thermo_lib_list:
    library_to_add = thermo_database.libraries[thermo_lib]
    merge_thermo_lib(base_lib, library_to_add)
    
# Save the libs
base_lib.save(base_lib.label)
for thermo_lib in thermo_lib_list:
    lib_to_add = thermo_database.libraries[thermo_lib]
    lib_to_add.save(thermo_lib)

[OPTIONAL] Remove tags

In [ ]:
for thermo_lib in thermo_lib_list:
    lib_to_mod = thermo_database.libraries[thermo_lib]
    for spc in lib_to_mod.entries.values():
        spc.shortDesc = ''
lib_to_mod.save(thermo_lib)